# PCA(主成分分析)を用いた異常検知について

## 基本アイデア
PCAで得られる低次元空間（主成分空間）には、データの主要な分散（情報）が集約される。一方、異常なデータはこの主成分空間にうまく収まらず、主成分に投影した際に大きな誤差（再構成誤差）が生じる。この「再構成誤差」を異常スコアとして利用し、一定の閾値を超える場合を異常値として検出する。

## 適用可能データ
* 数値データ（特に多次元データ）
* 継続的に監視できるセンサーデータ
* 時系列データ（ただし前処理が必要な場合あり）
* クレジットカード取引データやネットワークログデータのようなパターンを持つデータ

## メリット
* シンプルで解釈性が高い
* 非ラベルデータに対応
    * ラベルなしのデータでも適用可能で、異常検知に適している
* 次元削減の利点:
    * 高次元データを効率的に扱えるため、ノイズや冗長な特徴を削減できる

## デメリット
* 線形性の前提
    * データが**非線形**は関係を持つ場合、再構成誤差がうまく異常を反映しない
        * なので、分布が2つ以上あるようなデータ群に対してはPCAは合わない
* 閾値の決定
    * 再構成誤差の閾値を適切に設定するには、ドメイン知識や経験が必要
* 計算コスト
    * 高次元かつ大量データでは、PCAの計算コストが増大
* 動的なデータへの対応
    * 時間とともに変化するデータセットでは、PCAモデルの再訓練が必要




## 計算手順
### 1. データの前処理
* **データのスケーリング**: StandardScaler(平均0、分散1)にデータを標準化
* **欠損値の処理**: 補完または除外

### 2. PCAによる次元削減
1. データ行列$X$を用意
2. 共分散行列を計算する
$$
\Sigma = \frac{1}{n} X^T X
$$

3. 共分散行列の固有値と固有ベクトルを計算し、固有値が大きい順に主成分を選択する。

### 3. データの再構成
* 元のデータを主成分空間に射影する。
$$
X_{\text{projected}} = X W
$$
    * $W$は主成分ベクトルの行列
* 射影されたデータを元の空間に戻す。
$$
X_{\text{reconstructed}} = X_{\text{projected}} W^T
$$

### 4. 再構成誤差の計算
再構成誤差は、元のデータと再構成データの差で定義される。
$$
\text{Reconstruction Error} = \| X - X_{\text{reconstructed}} \|^2
$$

### 5. 閾値による異常検知
* 再構成誤差が大きいデータを異常値とみなす
* 閾値の設定方法:
    * 統計的手法（例えば、再構成誤差が平均±3σを超える場合など）
    * ROC曲線を用いた最適閾値の探索
    


## PCAにおける「線形性の前提」について
PCA(主成分分析)では、基本的にデータが**線形関係を持つ**ことが前提になっている

### 主成分分析における線形性の前提
#### PCAの数学的背景
1. **次元削減の仕組み**:
    * PCAは、データの分散が最大となる方向（主成分）を探索する。
    * 主成分は、元の特徴空間における線型結合（加重和）として表現される。
$$
z_i = w_1 x_1 + w_2 x_2 + \dots + w_n x_n
$$
    * $z_i$は主成分、$x_1, x_2, \dots, x_n$は元の特徴、$w_i$は重み

2. **線形性の仮定**:
    * PCAは、データの分散を説明するために直線的な射影を使用する。このため、元の特徴間に線形関係が強い場合に最も効果を発揮する。
    * 非線形な関係（例えば円や曲線など）は、PCAでは効果的に捉えることができない。

#### 線形性が前提となる理由
* PCAの計算プロセス（共分散行列の分解）線形代数に基づいており、非線形なパターンや特徴をモデル化する仕組みがない。
* 主成分は直線的な方向として定義されるため、複雑な曲線的関係を捉えるには不十分。

---

### 非線形データに対するPCAの限界

#### 例：非線形データの問題
* **円形の分布**:
    * 元のデータが円形（例えば2次元で$x^2 + y^2 = 1$）の場合、PCAはその構造を捉えられない。
    * PCAでは、この円形分布を2つの直線方向に分解するが、これは非線形な構造の本質的な特徴を反映していない。

#### 再構成誤差の問題
非線形データの場合、PCAによる再構成誤差は大きくなりがちで、異常検知の精度が低下する可能性がある。例えば、非線形データの異常をPCAで検出しようとしても、異常が主成分空間内に収まる場合、検出が困難になる。

---

### 非線形を扱う方法
PCAの線形性の限界を克服するために、非線形構造を捉える以下の拡張手法がある

1. **カーネルPCA（Kernel PCA）**
    * カーネルPCAは、データを高次元の特徴空間に非線形マッピングし、その空間でPCAを適用する。
    * カーネル関数（例：ガウシアンカーネル、ポリノミアルカーネル）を使用して非線形関係をモデル化する。
    * 適用例:
        * 円形や曲線的なデータの異常検知

2. **t-SNE (t-Distributed stochastic Neighbor Embedding)**
    * t-SNEは非線形次元削減手法で、データの局所的な構造を保持しながら低次元空間にマッピングする
    * 主にデータの可視化に使われるが、異常検知にも応用可能。

3. **オートエンコーダー**:
    * オートエンコーダーはニューラルネットワークを用いた次元削減手法で、非線形な関係をモデル化できる。
    * 再構成誤差を異常スコアとして利用する点ではPCAと似ているが、非線形関係にも対応可能。

4. **ISOMAP**:
    * グラフベースの次元削減手法で、データの非線形構造（多様体）を保持する。
    * 例えば、曲がりくねった表面上のデータ分布を効率的に表現可能。

---

### 線形PCAが適切なケース

* 特徴間に線形関係が強い場合
* データの次元が比較的低い場合（数十次元以下）
* 再構成誤差を用いた異常検知が主な目的の場合

### 非線形次元削減が必要なケース
* データに非線形なパターン（曲線的な構造や複雑な分布）が含まれる場合
* 高次元かつ複雑なデータを扱う場合
* 線形PCAで十分な再構成精度が得られない場合



## 複数の分布を持つデータへのPCA適用の問題点

### 1. 分布全体を1つの空間に投影する性質
* PCAはデータ全体の分散を考慮し、主成分を計算する。このため、複数の異なる分布を持つデータ群では、それぞれの分布が持つ固有の特徴を捉えにくい場合がある。
* 例えば以下の場合:
    * クラスごとに異なる方向性も持つデータ
    * 異なる分布をまたがる異常（境界にある点や全体に分散する異常点）。

 ### 2. 主成分のバイアス
 * 主成分は全データにおける「最大分散の方向」を基に選択される。このため、大きな分布を持つクラスタにバイアスがかかり、小さいな分布やまれな分布が無視される可能性がある。

 ### 3. 再構成誤差が異なる分布で同等に機能しない
    * 各分布に異なる特徴（スケールや形状）がある場合、再構成誤差がある分布では異常と見做されるが、別の分布では正常と見做される可能性がある。

## PCAが適切に機能する条件
1. **各分布が線形である場合**
    * 各分布が互いに近い線形関係を持つか、十分に分離されていれば、PCAは適切に機能することがある。
2. **分布間の分離が明確である場合**
    * 分布間が十分に離れている場合、PCAによる次元削減でそれぞれの分布の主成分がうまく抽出される場合がある。
3. **データの変換・前処理が適切に行われている場合**
    * 分布間の違いを抑えるため、データのスケーリングやクラスタごとの処理が有効。


## 改善手法と代替手法
1. **クラスタリングと組み合わせる**
    * データに複数の分布がある場合、クラスタリングを事前に行い、各クラスタに対して個別にPCAを適用することで、分布ごとの特徴を考慮した異常検知が可能になる。
    * 例：K-meansクラスタリングを使用してクラスタに分け、その後、各クラスタで再構成誤差を計算。
2. **カーネルPCAの活用**
    * 非線形分布や複数の分布をより効果的に扱うために、カーネルPCAを用いることができる。カーネルPCAでは、非線形な特徴を考慮しつつデータを射影できるため、複数分布の特性を保持できる。
3. **ガウス混合モデル (GMM)**
    * 分布が複数存在する場合、ガウス混合モデルを用いることで、データを複数の正規分布の線形結合としてモデリングできる。異常検知には、各データ点の混合分布への尤度（ログ尤度）を利用する。
4. **ロバストPCA (Robust PCA)**
    * ノイズや異常値に強い次元削減手法であるロバストPCAを使用することで、複数分布の影響を低減しつつ異常検知を行える。
5. **クラスタごとの再構成誤差の調整**
    * CAを適用した後、クラスタごとに再構成誤差を標準化して異常スコアを計算する方法。このアプローチにより、クラスタ間の再構成誤差の違いを補正できる。
6. **ディープラーニングを用いたアプローチ**
    * オートエンコーダを使用すると、複雑な分布間の非線形関係を学習でき、PCAよりも柔軟な異常検知が可能。
